Rank the passages using chosen score.

In [13]:
import json
def read_rows(path):
    rows = []
    for line in open(path):
        rows.append(json.loads(line))
    return rows

In [14]:
paths_dataset = ['test-B-big/wiki.jl', 'test-B-big/legal.jl', 'test-B-big/allegro.jl']
base_path = 'test-B-big/out/full_with_chatgpt/'
paths_out = [base_path+'wiki.tsv', base_path+'legal.tsv', base_path+'allegro.tsv']

In [15]:
score_by = 'log_reg_full_with_chatgpt'

In [16]:
def get_best_passages(scores, k=10):
    best_passages = []
    for score, passage_id in sorted(scores):
        if passage_id not in best_passages:
            best_passages.append(passage_id)
        if len(best_passages) == k:
            break
    return best_passages

In [17]:
for path_dataset, path_out in zip(paths_dataset, paths_out):
    rows = read_rows(path_dataset)
    question_id = rows[0]['question_id']
    output_ids = []
    scores = []
    for row in rows:
        if row['question_id'] != question_id:
            output_ids.append(get_best_passages(scores))
            scores = []
            question_id = row['question_id']
        scores.append((-row[score_by], row['passage_id']))
    output_ids.append(get_best_passages(scores))
    
    f_out = open(path_out, 'w')
    for passage_ids in output_ids:
        f_out.write('\t'.join(passage_ids)+'\n')
    f_out.close()

In [18]:
import math
def score_passages(predicted_path, expected_path):
    score = 0
    for predicted_line, expected_line in zip(open(predicted_path).read().split('\n')[:-1], open(expected_path).read().split('\n')[:-1]):
        predicted_ids = predicted_line.split()
        expected_ids = expected_line.split()

        dcg = 0
        for i, id in enumerate(predicted_ids):
            if id in expected_ids:
                dcg += 1/math.log2(i+2)
        idcg = sum([1/math.log2(i+2) for i in range(len(expected_ids))])

        score += dcg/idcg
    return 100*score/(len(open(predicted_path).read().split('\n'))-1)